In [15]:
import pandas as pd
import os
import numpy as np
from Levenshtein import distance as levenshtein_distance
from collections import Counter
import glob

In [16]:
# Set these so that basefolder is the folder containing the raw data folder
# and outfolder being the folder to save the cleaned data in
basefolder = ''
outfolder = ''

In [17]:
def find_NN_dist(test_seq, reference_df, column):
    reference_df['dist'] = reference_df[column].apply(lambda x: levenshtein_distance(x, test_seq))
    NNdist = min(reference_df['dist'])
    return NNdist

In [18]:
# Find all sequencing errors
read_limit = 10
to_trash=pd.DataFrame() # Dataframe to collect sequences that will be excluded
overlapping_clones = pd.DataFrame()
for folder in ['GCA_gDNA']:#, 'Agematched Controls', 'other Controls']: #'GCA_FFPE', 
    print(folder)
    # Find all files with name pattern _clones_inclNN.txt in folder
    for filename in glob.glob(os.path.join(basefolder, 'raw data', folder, '*_clones_inclNN.txt')):
        sample = pd.read_csv(
                filename,
                delimiter='\t',
                usecols=['cloneCount', 'cloneFraction', 'aaSeqCDR3','nSeqCDR3', 'bestVHit', 'bestJHit']
            )
        # Extract patient id from filename
        sample['Name'] = [filename.split('/')[-1].split('_')[0] for i in range(len(sample))]
        # Remove all singletons and doubles
        sample = sample[sample['cloneCount'] >= 2]
        # Remove seqs with less than 10 reads if they are only one nucleotide away from another seq
        pot_errors = sample[sample['cloneCount'] <= read_limit]
        ref = sample[sample['cloneCount'] > read_limit]
        # find distance to closest neighbor in higher read category
        pot_errors['NNdist'] = pot_errors['nSeqCDR3'].apply(lambda x: find_NN_dist(x, ref, 'nSeqCDR3'))
        # find those with less than 2 nucleotides distance
        pot_errors = pot_errors[pot_errors['NNdist'] < 2]
        # save to trash later
        to_trash = pd.concat([to_trash, pot_errors])
        # add all clones in the sample as potentially overlapping clones
        overlapping_clones = pd.concat(
                        [overlapping_clones, sample],
                        ignore_index=True
                    )
    
    count_dict = Counter(overlapping_clones['nSeqCDR3'])
    # exclude all clones that occur only once (these dont overlap between at least 2 patients)
    exclude1 = [x for x in count_dict if count_dict[x] < 2]
    # result is a dataframe with all clones that appear in at least two patients
    overlapping_clones = overlapping_clones[~overlapping_clones['nSeqCDR3'].isin(exclude1)]

to_trash_RE = to_trash.dropna(subset=['NNdist'])
to_trash_RE.to_csv(os.path.join(basefolder, 'to_trash_sequencing_errors.csv'))

GCA_gDNA


/Users/wbr/opt/anaconda3/envs/leeds/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/wbr/opt/anaconda3/envs/leeds/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [19]:
### Find all Sequence Sharing Artifacts Cohort-Wise (Only GCA data)
# uncomment one of the sections below

### Cohort definitions for gDNA samples
### set Cohort = 0 for controls, 1 for cohort 1 GCA data, 2 for cohort 2 GCA data

'''
overlapping_clones['Cohort'] = [2 if 'gDNA' in x else 0 for x in overlapping_clones['Name']]
cohort1 = ['14907-gDNA','14915-gDNA','14914-gDNA','14910-gDNA','14922-gDNA',
'14912-gDNA','14927-gDNA','14930-gDNA','14929-gDNA','14931-gDNA','14926-gDNA',
'16094-gDNA','12898-gDNA','12849-gDNA','12902-gDNA','12835-gDNA','12223-gDNA',
'11037-gDNA','12954-gDNA','15272-gDNA','15429-gDNA','15456-gDNA','15518-gDNA','15576-gDNA','15579-gDNA']
overlapping_clones['Cohort'] = [x['Cohort'] if x['Name'] not in cohort1 else 1 for i,x in overlapping_clones.iterrows()]
'''
### Cohort definitions for FFPE samples
### set Cohort = 0 for controls, 1 for cohort 1 GCA data, 2 for cohort 2 GCA data
'''
overlapping_clones['Cohort'] = [2 if 'FFPE' in x else 0 for x in overlapping_clones['Name']]
cohort1 = ['14907-FFPE','14915-FFPE','14914-FFPE','14910-FFPE','14922-FFPE',
'14912-FFPE','14927-FFPE','14930-FFPE','14929-FFPE','14931-FFPE','14926-FFPE',
'16094-FFPE','12898-FFPE','12849-FFPE','12902-FFPE','12835-FFPE','12223-FFPE',
'11037-FFPE','12954-FFPE','15272-FFPE','15429-FFPE','15456-FFPE','15518-FFPE','15576-FFPE','15579-FFPE']
overlapping_clones['Cohort'] = [x['Cohort'] if x['Name'] not in cohort1 else 1 for i,x in overlapping_clones.iterrows()]
'''

# find overlapping sequences
to_trash_overlap = pd.DataFrame()

for cohort in [0,1,2]:
    #extract only overlapping sequences from the current cohort
    cohort_data = overlapping_clones[overlapping_clones['Cohort'] == cohort]
    unique_overlapping_clones = set(cohort_data['nSeqCDR3'])

    for clone in unique_overlapping_clones:
        # find all instances of a clone
        subset = cohort_data[cohort_data['nSeqCDR3'] == clone]
        # calculate expansion in relation to largest expanded occurences of that clone
        largest_clone = np.max(subset['cloneCount'])
        subset['fraction'] = [x/largest_clone for x in subset['cloneCount']]
        # remove seq if identical NN seq in other sample with higher clone count (fraction  < 0.1 )
        to_trash_overlap = pd.concat([to_trash_overlap, subset[subset['fraction'] < 0.01]])

to_trash = pd.concat([to_trash_overlap, to_trash_RE])
to_trash['UI'] = [x['nSeqCDR3']+str(x['cloneCount']) for i,x in to_trash.iterrows()]

/Users/wbr/opt/anaconda3/envs/leeds/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
### Go through data again, trash all the trash and save new files
for folder in ['GCA_gDNA', 'Agematched Controls', 'other Controls']: #'GCA_FFPE'
    #print(folder)
    for filename in glob.glob(os.path.join(basefolder, 'raw data', folder, '*_clones_inclNN.txt')):
        #print(filename)
        sample = pd.read_csv(
                filename,
                delimiter='\t',
                usecols=['cloneCount', 'cloneFraction', 'aaSeqCDR3','nSeqCDR3', 'bestVHit', 'bestJHit']
            )
        name = filename.split('/')[-1].split('_')[0]
        # Retain only sequences with count of at least 2
        sample = sample[sample['cloneCount'] >= 2]
        sample['UI'] = [x['nSeqCDR3']+str(x['cloneCount']) for i,x in sample.iterrows()]
        # remove everything that we identified as trash before
        trash_nseqs = to_trash[to_trash['Name'] == name]['UI']
        sample = sample[~sample['UI'].isin(trash_nseqs)]
        sample = sample.drop(columns=['UI'])
        # save clean data
        #sample.to_csv(os.path.join(outfolder, name+'_cleaned_2.txt'), index=False)